# Timeline

<img src="https://i.ibb.co/KKVVDjW/Image-Tittle.jpg" alt="Image-Tittle" border="0">

<img src="https://i.ibb.co/HKtKPy6/announcments.jpg" alt="announcments" border="0">

# Declaration

* According to MoHFW https://www.mohfw.gov.in/ some new cases have been reassigned states as per latest information
* Confirmed cases includes both Indian and Foriegn citizen
* Cured cases includes cured, discharged and migrated patients 

## Data Source
* https://www.mohfw.gov.in/


### GitHub Repository for this Notebook
<a href="https://github.com/sreyaz01/covid-19-india-data-analysis"><img src="https://i.ibb.co/B3vvzTy/0-s-Y-XTIBzlfd2zskq.png" alt="0-s-Y-XTIBzlfd2zskq" border="0" width="300" height="100"></a>

In [1]:
#!conda install -c conda-forge cufflinks-py

# Libraries

In [2]:
## utility libraries
from IPython.core.display import HTML
from datetime import datetime
from datetime import timedelta


# storing and anaysis
import pandas as pd
import geopandas as gpd
import numpy as np

#Visualization Libraries
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from plotly.offline import init_notebook_mode,plot,iplot

import folium
import seaborn as sns

import cufflinks as cf

# Warning
import warnings
warnings.filterwarnings('ignore')


print('Pandas Version' , pd.__version__)
print('Matplotlib Version' , matplotlib.__version__)
print('Plotly Version' , plotly.__version__)
print('Seaborn Version' , sns.__version__)

C:\Users\sreya\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



Pandas Version 1.0.1
Matplotlib Version 3.2.1
Plotly Version 4.5.4
Seaborn Version 0.9.0


In [3]:
# setting up some setting for libraries
%matplotlib inline
plt.rcParams['figure.figsize'] = 17,8
pyo.init_notebook_mode(connected=True)
cf.go_offline()

#style.use('ggplot')

In [4]:
# color pallette
cnf = '#393e46' # confirmed - grey
dth = '#ff2e63' # death - red
rec = '#21bf73' # recovered - cyan
act = '#fe9801' # active case - yellow

# Dataset 

In [5]:
%ls C:\Users\sreya\Desktop\Pythone_File\COVID-19_INDIA\input\covid19-corona-virus-india-dataset

 Volume in drive C has no label.
 Volume Serial Number is 0821-B132

 Directory of C:\Users\sreya\Desktop\Pythone_File\COVID-19_INDIA\input\covid19-corona-virus-india-dataset

19-04-2020  12:54    <DIR>          .
19-04-2020  12:54    <DIR>          ..
23-04-2020  14:30             8,397 cases_over_time_flourish.csv
23-04-2020  14:30            61,349 complete.csv
10-04-2020  16:57         1,219,997 patients_data.csv
               3 File(s)      1,289,743 bytes
               2 Dir(s)  199,740,362,752 bytes free


In [6]:
#importing data
df = pd.read_csv(r'C:\Users\sreya\Desktop\Pythone_File\COVID-19_INDIA\input\covid19-corona-virus-india-dataset\complete.csv',
                parse_dates = ['Date'])

df.tail()

,Date,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Latitude,Longitude,Death,Total Confirmed cases
1202,2020-04-23,Telengana,0,0,194,18.1124,79.0193,23,945
1203,2020-04-23,Tripura,0,0,1,23.9408,91.9882,0,2
1204,2020-04-23,Uttar Pradesh,0,0,173,26.8467,80.9462,21,1449
1205,2020-04-23,Uttarakhand,0,0,23,30.0668,79.0193,0,46
1206,2020-04-23,West Bengal,0,0,79,22.9868,87.8550,15,456


In [7]:
df.columns

Index(['Date', 'Name of State / UT', 'Total Confirmed cases (Indian National)',
       'Total Confirmed cases ( Foreign National )',
       'Cured/Discharged/Migrated', 'Latitude', 'Longitude', 'Death',
       'Total Confirmed cases'],
      dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1207 entries, 0 to 1206
Data columns (total 9 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   Date                                        1207 non-null   datetime64[ns]
 1   Name of State / UT                          1207 non-null   object        
 2   Total Confirmed cases (Indian National)     1207 non-null   int64         
 3   Total Confirmed cases ( Foreign National )  1207 non-null   int64         
 4   Cured/Discharged/Migrated                   1207 non-null   int64         
 5   Latitude                                    1207 non-null   float64       
 6   Longitude                                   1207 non-null   float64       
 7   Death                                       1207 non-null   int64         
 8   Total Confirmed cases                       1207 non-null   int64         
dtypes: dateti

## State Map data

In [9]:
# shape files
map_data = gpd.read_file(r'C:\Users\sreya\Desktop\Pythone_File\COVID-19_INDIA\input\india-district-wise-shape-files\output.shp')

map_data.head()

,objectid,statecode,statename,state_ut,distcode,distname,distarea,totalpopul,totalhh,totpopmale,totpopfema,st_areasha,st_lengths,geometry
0,1,05,Uttarakhand,STATE,066,Nainital,4251.0,954605.0,191383.0,493666.0,460939.0,5.322546e+09,506182.695952,"POLYGON ((79.52659 29.05543, 79.52550 29.05545..."
1,2,05,Uttarakhand,STATE,060,Dehradun,3088.0,1696694.0,347001.0,892199.0,804495.0,4.177236e+09,578188.681639,"POLYGON ((77.87557 30.26052, 77.87467 30.26087..."
2,3,05,Uttarakhand,STATE,064,Almora,3144.0,622506.0,140577.0,291081.0,331425.0,4.140751e+09,463454.225766,"POLYGON ((79.28494 29.92735, 79.28495 29.92723..."
3,4,05,Uttarakhand,STATE,065,Champawat,1766.0,259648.0,53953.0,131125.0,128523.0,2.294297e+09,314508.010612,"POLYGON ((80.12479 29.01308, 80.12481 29.01306..."
4,5,05,Uttarakhand,STATE,056,Uttarkashi,8016.0,330086.0,67602.0,168597.0,161489.0,1.085166e+10,786425.588972,"POLYGON ((78.92267 31.25333, 78.93106 31.26840..."


In [10]:
#State Wise Grouping Data
states = map_data.dissolve(by='statename').reset_index()
states.head()

,statename,geometry,objectid,statecode,state_ut,distcode,distname,distarea,totalpopul,totalhh,totpopmale,totpopfema,st_areasha,st_lengths
0,Andaman & Nicobar Islands,"MULTIPOLYGON (((93.84419 7.22173, 93.84565 7.2...",380,35,UNION TERRITORY,639,North & Middle Andaman,3736.0,105597.0,26199.0,54861.0,50736.0,3.582828e+09,1.665121e+06
1,Andhra Pradesh,"MULTIPOLYGON (((82.36333 16.71061, 82.35167 16...",134,28,STATE,549,Prakasam,17626.0,3397448.0,860463.0,1714764.0,1682684.0,1.923605e+10,8.177556e+05
2,Arunachal Pradesh,"POLYGON ((95.48591 26.75778, 95.48534 26.75821...",404,12,STATE,259,Lohit,5212.0,145726.0,30005.0,76221.0,69505.0,5.238221e+09,4.149939e+05
3,Assam,"POLYGON ((89.83741 26.41240, 89.83771 26.41230...",420,18,STATE,302,Goalpara,1824.0,1008183.0,198454.0,513292.0,494891.0,2.452588e+09,3.629872e+05
4,Bihar,"POLYGON ((85.41963 24.54472, 85.41772 24.54562...",224,10,STATE,228,Sheikhpura,689.0,636342.0,100701.0,329743.0,306599.0,6.371338e+08,1.976915e+05


In [11]:
states['statename'] = states['statename'].str.replace('&', 'and')
states['statename'] = states['statename'].str.replace('NCT of ', '')
states['statename'] = states['statename'].str.replace('Chhatisgarh', 'Chhattisgarh')
states['statename'] = states['statename'].str.replace('Orissa', 'Odisha')
states['statename'] = states['statename'].str.replace('Pondicherry', 'Puducherry')

states['statename'].unique()

array(['Andaman and Nicobar Islands', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh',
       'Chhattisgarh', 'Dadra and Nagar Haveli', 'Daman and Diu', 'Goa',
       'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jammu and Kashmir',
       'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh', 'Lakshadweep',
       'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram',
       'Delhi', 'Nagaland', 'Odisha', 'Puducherry', 'Punjab', 'Rajasthan',
       'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh',
       'Uttarakhand', 'West Bengal'], dtype=object)

# Preprocessing


## Cleaning

In [12]:
df_clean = df[['Date', 'Name of State / UT', 'Latitude', 'Longitude', 'Total Confirmed cases', 'Death', 'Cured/Discharged/Migrated']]
df_clean.columns = ['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Deaths', 'Cured']

df_clean['Date'] = df_clean['Date'].dt.date

df_clean['Active'] = df_clean['Confirmed']-(df_clean['Deaths']+df_clean['Cured'])
df_clean['Mortality Rate'] = df_clean['Deaths']/df_clean['Confirmed']
df_clean['Recovery Rate'] = df_clean['Cured']/df_clean['Confirmed']
df_clean.tail()

,Date,State/UT,Latitude,Longitude,Confirmed,Deaths,Cured,Active,Mortality Rate,Recovery Rate
1202,2020-04-23,Telengana,18.1124,79.0193,945,23,194,728,0.024339,0.205291
1203,2020-04-23,Tripura,23.9408,91.9882,2,0,1,1,0.000000,0.500000
1204,2020-04-23,Uttar Pradesh,26.8467,80.9462,1449,21,173,1255,0.014493,0.119393
1205,2020-04-23,Uttarakhand,30.0668,79.0193,46,0,23,23,0.000000,0.500000
1206,2020-04-23,West Bengal,22.9868,87.8550,456,15,79,362,0.032895,0.173246


## Finding Latest Data Only

In [13]:
latest = df_clean[df_clean['Date']==max(df_clean['Date'])]


total_confirm = latest['Confirmed'].sum()
total_active = latest['Active'].sum()
total_cured = latest['Cured'].sum()
total_death = latest['Deaths'].sum()


now  = datetime.now().strftime("%B %d, %Y")

print(u"\u2022",f'Total Number of Confirmed Covid 2019 Cases across India till date ({now}):', total_confirm)
print(u"\u2022",f'Total Number of Active Cases till date ({now}):', total_active)
print(u"\u2022",f'Total Number of Cured Cases across India till date ({now}):', total_cured)
print(u"\u2022",f'Total Number of Cured Cases across India till date ({now}):', total_death)

• Total Number of Confirmed Covid 2019 Cases across India till date (April 23, 2020): 21393
• Total Number of Active Cases till date (April 23, 2020): 16454
• Total Number of Cured Cases across India till date (April 23, 2020): 4258
• Total Number of Cured Cases across India till date (April 23, 2020): 681


In [14]:
tm = latest.melt(id_vars="Date", value_vars=['Active', 'Deaths', 'Cured'])
tm.head()

,Date,variable,value
0,2020-04-23,Active,7
1,2020-04-23,Active,669
2,2020-04-23,Active,0
3,2020-04-23,Active,15
4,2020-04-23,Active,95


In [15]:
fig = px.treemap(tm, path=["variable"], values="value",height=250, width=800,
                 color_discrete_sequence=[act, rec, dth], title='Latest Stats')

fig.data[0].textinfo = 'label+value+text'
fig.show()

In [16]:
latest.head()

,Date,State/UT,Latitude,Longitude,Confirmed,Deaths,Cured,Active,Mortality Rate,Recovery Rate
1175,2020-04-23,Andaman and Nicobar Islands,11.7401,92.6586,18,0,11,7,0.000000,0.611111
1176,2020-04-23,Andhra Pradesh,15.9129,79.7400,813,24,120,669,0.029520,0.147601
1177,2020-04-23,Arunachal Pradesh,28.2180,94.7278,1,0,1,0,0.000000,1.000000
1178,2020-04-23,Assam,26.2006,92.9376,35,1,19,15,0.028571,0.542857
1179,2020-04-23,Bihar,25.0961,85.3131,143,2,46,95,0.013986,0.321678


In [17]:
temp = latest.groupby(by = ['State/UT']).sum()

temp.tail()

,Latitude,Longitude,Confirmed,Deaths,Cured,Active,Mortality Rate,Recovery Rate
State/UT,,,,,,,,
Telengana,18.1124,79.0193,945,23,194,728,0.024339,0.205291
Tripura,23.9408,91.9882,2,0,1,1,0.000000,0.500000
Uttar Pradesh,26.8467,80.9462,1449,21,173,1255,0.014493,0.119393
Uttarakhand,30.0668,79.0193,46,0,23,23,0.000000,0.500000
West Bengal,22.9868,87.8550,456,15,79,362,0.032895,0.173246


In [18]:
temp = temp[['Confirmed','Deaths','Cured','Active','Mortality Rate','Recovery Rate']]
temp.sort_values('Confirmed',ascending=False,inplace = True)
#temp.head()

temp.style\
    .background_gradient(cmap="Blues", subset=['Active','Confirmed'])\
    .background_gradient(cmap="Greens", subset=['Cured', 'Recovery Rate'])\
    .background_gradient(cmap="Reds", subset=['Deaths', 'Mortality Rate'])

,Confirmed,Deaths,Cured,Active,Mortality Rate,Recovery Rate
State/UT,,,,,,
Maharashtra,5652,269,789,4594,0.047594,0.139597
Gujarat,2407,103,179,2125,0.042792,0.074366
Delhi,2248,48,724,1476,0.021352,0.322064
Rajasthan,1890,27,230,1633,0.014286,0.121693
Tamil Nadu,1629,18,662,949,0.011050,0.406384
Madhya Pradesh,1592,80,148,1364,0.050251,0.092965
Uttar Pradesh,1449,21,173,1255,0.014493,0.119393
Telengana,945,23,194,728,0.024339,0.205291
Andhra Pradesh,813,24,120,669,0.029520,0.147601


In [19]:
temp.columns

Index(['Confirmed', 'Deaths', 'Cured', 'Active', 'Mortality Rate',
       'Recovery Rate'],
      dtype='object')

In [20]:
#Visualization
temp_1 = temp[['Confirmed', 'Deaths', 'Cured']]

temp_1.iplot(kind = 'bar',xTitle= 'State/UT' , yTitle='Numbers of Cases',mode = 'markers+lines',
            title = f'Cases State Wise on {now}')

In [21]:
temp_2 = temp[['Mortality Rate','Recovery Rate']]
               
temp_2.iplot(kind ='scatter',xTitle='State/UT',yTitle='Avrage',title = f'Mortality and Recovery Rate on {now}',
             mode = 'markers', size = 5)

In [22]:
# Date wise data visualization whole country

temp = df_clean.groupby(by = ['Date']).sum()
temp.drop(['Latitude','Longitude','Mortality Rate','Recovery Rate'],axis=1,inplace=True)

temp.tail()

,Confirmed,Deaths,Cured,Active
Date,,,,
2020-04-19,16116,519,2302,13295
2020-04-20,17265,543,2547,14175
2020-04-21,18601,590,3252,14759
2020-04-22,18985,603,3260,15122
2020-04-23,21393,681,4258,16454


In [23]:
temp.iplot(title = 'Covid-19 Growth in India', yTitle='Cases',size=5,mode='markers+lines')

## No. of New Cases EveryDay After 50

In [24]:
#df_clean.columns

In [45]:
cases_df = df_clean.groupby('Date')['Confirmed', 'Deaths'].sum()

filt_cnf = (cases_df['Confirmed'] >= 50)

temp = cases_df[filt_cnf].diff().dropna()

col_y = ['Confirmed','Deaths']
colr = [cnf,dth]

for i,x in enumerate(col_y):
    temp.iplot(kind = 'scatter',mode = "markers+lines" ,size = 5,y = col_y[i],color=colr[i],yTitle=col_y[i],title=f'New {col_y[i]} Cases after Crossing 50 Confirmed Cases')

## Top 10 States

In [46]:
top_10 = latest.groupby(by = ['State/UT']).agg({'Confirmed': 'sum', 'Deaths': 'sum', 'Cured' : 'sum', 'Active' : 'sum'})\
                .nlargest(10,['Confirmed','Deaths','Cured','Active'])

top_10

,Confirmed,Deaths,Cured,Active
State/UT,,,,
Maharashtra,5652,269,789,4594
Gujarat,2407,103,179,2125
Delhi,2248,48,724,1476
Rajasthan,1890,27,230,1633
Tamil Nadu,1629,18,662,949
Madhya Pradesh,1592,80,148,1364
Uttar Pradesh,1449,21,173,1255
Telengana,945,23,194,728
Andhra Pradesh,813,24,120,669


In [47]:
#Creating Figures
plot_c = px.bar(top_10.sort_values('Confirmed') ,x="Confirmed",y = top_10.sort_values('Confirmed').index,
               text='Confirmed', orientation='h', color_discrete_sequence = [cnf])

plot_d = px.bar(top_10.sort_values('Deaths'),x="Deaths",y = top_10.sort_values('Deaths').index,
               text='Deaths', orientation='h', color_discrete_sequence = [dth])

plot_r = px.bar(top_10.sort_values('Cured'),x="Cured",y = top_10.sort_values('Cured').index,
               text='Cured', orientation='h', color_discrete_sequence = [rec])

plot_a = px.bar(top_10.sort_values('Active'),x="Active",y = top_10.sort_values('Active').index,
               text='Active', orientation='h', color_discrete_sequence = [act])


# plot
fig = make_subplots(rows=2, cols=2, shared_xaxes=False, horizontal_spacing=0.14, vertical_spacing=0.08,
                    subplot_titles=('Confirmed cases', 'Deaths reported', 'Recovered', 'Active cases'))

fig.add_trace(plot_c['data'][0],row=1, col=1)
fig.add_trace(plot_d['data'][0],row=1, col=2)
fig.add_trace(plot_r['data'][0],row=2, col=1)
fig.add_trace(plot_a['data'][0],row=2, col=2)

fig.update_layout(height=600 ,title_text="Top 10 States ")


# Cases Rises in State Over Time

https://app.flourish.studio/visualisation/1977187/edit

In [48]:
HTML('<div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/1977187" data-url="https://flo.uri.sh/visualisation/1977187/embed"><script src="https://public.flourish.studio/resources/embed.js"></script></div>')

In [49]:
df_clean.columns

Index(['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Deaths',
       'Cured', 'Active', 'Mortality Rate', 'Recovery Rate'],
      dtype='object')

In [50]:
#Deaths,Cured ,Active Cases Date Wise

col = ['Deaths','Cured','Active']

for i,val in enumerate(col):
    p_df = pd.pivot_table(df_clean,index  = 'Date', values = val, columns ='State/UT').fillna(0).astype('int').reset_index()
    p_df.iplot(x = 'Date' ,title = col[i], xTitle = 'Date',yTitle = 'Cases')

# Geographical Map

In [51]:
latest.columns

Index(['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Deaths',
       'Cured', 'Active', 'Mortality Rate', 'Recovery Rate'],
      dtype='object')

In [52]:
geo_map = folium.Map([20.5937,78.9629],zoom_start=4,tiles ='cartodbpositron' )

for lat,long,active,deaths,cured,name in zip(latest['Latitude'],latest['Longitude'],\
                                             latest['Active'],latest['Deaths'],\
                                             latest['Cured'],latest['State/UT']):

    folium.CircleMarker([lat,long],radius=active*0.03\
                       ,tooltip = (f'''<strong>name</strong>: {str(name).capitalize()} <br>
                               <strong>Active</strong>: {str(active)}<br>
                               <strong>Deaths</strong>: {str(deaths)}<br>
                               <strong>Cured</strong>: {str(cured)}<br>''')\
                       ,color = 'red',fill_color = 'red',fill_opacity=0.3).add_to(geo_map)

geo_map

In [33]:
def fun(list):
    for x in list:
        b =str(x)

fig = px.choropleth(df_clean, locations="State/UT", locationmode='country names', color=np.log(df_clean["Confirmed"]), 
                    hover_name="State/UT", animation_frame=map(fun,df_clean["Date"]),
                    title='Cases over time', color_continuous_scale=px.colors.sequential.Magenta)
fig.update(layout_coloraxis_showscale=False)
fig.show()

TypeError: object of type 'map' has no len()

In [ ]:
def fun(list):
    for x in list:
        b =str(x)
    
new = map(fun,df_clean["Date"])

